# Homework

## Dataset

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon). 

In [ ]:
!wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip | unzip dino-dragon.zip

## Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons. 

The dataset contains separate folders for training and test sets. 


## Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).


In [23]:
import numpy as np

from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [69]:
# create model
model = Sequential()
# add model layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [70]:
optimizer = SGD(learning_rate=0.002, momentum=0.8)

# compile model using accuracy to measure model performance
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

## Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

## Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

In [60]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

## Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [71]:
gen = ImageDataGenerator(rescale=1./255)

train_ds = gen.flow_from_directory(
    './train',
    class_mode='binary',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True
)

test_ds = gen.flow_from_directory(
    './test',
    class_mode='binary',
    target_size=(150, 150),
    batch_size=20,
    shuffle=False
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [72]:
# fit model
history = model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10
80/80 [==============================] - 6s 68ms/step - loss: 0.6636 - accuracy: 0.6255 - val_loss: 0.5789 - val_accuracy: 0.7868
Epoch 2/10
80/80 [==============================] - 5s 66ms/step - loss: 0.5364 - accuracy: 0.7440 - val_loss: 0.4922 - val_accuracy: 0.7563
Epoch 3/10
80/80 [==============================] - 5s 66ms/step - loss: 0.4398 - accuracy: 0.8112 - val_loss: 0.4076 - val_accuracy: 0.8198
Epoch 4/10
80/80 [==============================] - 5s 66ms/step - loss: 0.3726 - accuracy: 0.8532 - val_loss: 0.3597 - val_accuracy: 0.8401
Epoch 5/10
80/80 [==============================] - 5s 66ms/step - loss: 0.3423 - accuracy: 0.8551 - val_loss: 0.3360 - val_accuracy: 0.8503
Epoch 6/10
80/80 [==============================] - 5s 67ms/step - loss: 0.3076 - accuracy: 0.8752 - val_loss: 0.3639 - val_accuracy: 0.8299
Epoch 7/10
80/80 [==============================] - 5s 67ms/step - loss: 0.2641 - accuracy: 0.9028 - val_loss: 0.2980 - val_accuracy: 0.8782
Epoch 8/10
80

## Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

In [73]:
np.mean(history.history['accuracy'])

0.8432245850563049

## Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

In [74]:
np.std(history.history['loss'])

0.14442193899135886

## Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [75]:
gen_augm = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_augm_ds = gen_augm.flow_from_directory(
    './train',
    class_mode='binary',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True
)

Found 1594 images belonging to 2 classes.


## Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [76]:
# fit model
history = model.fit(train_augm_ds, validation_data=test_ds, epochs=20, initial_epoch=10)

Epoch 11/20
80/80 [==============================] - 20s 246ms/step - loss: 0.4769 - accuracy: 0.7735 - val_loss: 0.3651 - val_accuracy: 0.8376
Epoch 12/20
80/80 [==============================] - 10s 123ms/step - loss: 0.4396 - accuracy: 0.7836 - val_loss: 0.3648 - val_accuracy: 0.8249
Epoch 13/20
80/80 [==============================] - 10s 125ms/step - loss: 0.4192 - accuracy: 0.8112 - val_loss: 0.3509 - val_accuracy: 0.8274
Epoch 14/20
80/80 [==============================] - 10s 124ms/step - loss: 0.3854 - accuracy: 0.8363 - val_loss: 0.4868 - val_accuracy: 0.7741
Epoch 15/20
80/80 [==============================] - 10s 123ms/step - loss: 0.3923 - accuracy: 0.8181 - val_loss: 0.5067 - val_accuracy: 0.7690
Epoch 16/20
80/80 [==============================] - 10s 123ms/step - loss: 0.3625 - accuracy: 0.8407 - val_loss: 0.3398 - val_accuracy: 0.8680
Epoch 17/20
80/80 [==============================] - 10s 123ms/step - loss: 0.3529 - accuracy: 0.8494 - val_loss: 0.4214 - val_accuracy:

In [77]:
np.mean(history.history['val_loss'])

0.38906014859676363

## Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24


In [82]:
np.mean(history.history['val_accuracy'][5:])

0.8477157354354858